In [ ]:
!pip install loguru seaborn wonderwords tqdm
DATA_DIR = "../data/"

In [1]:
from skimage import io
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Subset

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [1]:
!wget https://www5.cs.fau.de/fileadmin/research/datasets/fundus-images/all.zip
!unzip -q -d ../data all.zip

--2023-05-02 18:00:54--  https://www5.cs.fau.de/fileadmin/research/datasets/fundus-images/all.zip
Resolving www5.cs.fau.de (www5.cs.fau.de)... 131.188.35.25
Connecting to www5.cs.fau.de (www5.cs.fau.de)|131.188.35.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76317613 (73M) [application/zip]
Saving to: ‘all.zip’

all.zip             100%[===================>]  72.78M  13.9MB/s    in 5.4s    

2023-05-02 18:01:00 (13.4 MB/s) - ‘all.zip’ saved [76317613/76317613]



In [ ]:
# only when using preprocessed data

In [ ]:
%%bash
(source_file="/content/drive/My Drive/advanced_image_analysis/super_resolution/data_ps512_str4.zip"
dest_dir="{DATA_DIR}"
if [ -d "$dest_dir" ]; then
  echo "Data already exists, skipping copy"
else
  cp "$source_file" .
  mkdir $dest_dir
  unzip -q -d ../data data_ps512_str4.zip
  echo "Data copied and unzipped successfully"
fi)

In [3]:
from pathlib import Path
from loguru import logger

from image_preprocessing import preprocess
from data import Data, init_save_dir
from srgan import UNet, VGG16Discriminator

PATCH_SIZE = 512
STRIDE = 4
# only for re-preprocessing the imagery (patch, crop, upsample)
DATA_DIR = "../data/"
INPUT_DIR = f"{DATA_DIR}/images"

run_preprocessing = True

In [5]:
!ls "{DATA_DIR}patches_512" | wc -l

    1920


In [ ]:
if run_preprocessing:
    preprocess(
        patch_size=PATCH_SIZE,
        stride=STRIDE,
        input_dir=INPUT_DIR,
        output_dir=DATA_DIR,
    )


patchify: 6it [00:03,  1.68it/s]

In [7]:
from config import PATCHES_TEMPLATE, CROPPED_TEMPLATE
from evaluate import compare_batches

patches_dir = Path(PATCHES_TEMPLATE.format(PATCH_SIZE))
cropped_dir = Path(CROPPED_TEMPLATE.format(PATCH_SIZE, STRIDE))

In [8]:
from skimage.metrics import peak_signal_noise_ratio, mean_squared_error
from image_preprocessing import get_cropped_path

h, w = 200, 300  # zoom 

patches_dir = DATA_DIR / Path(f"patches_{PATCH_SIZE}/")
cropped_dir = DATA_DIR / Path(f"cropped/ps{PATCH_SIZE}_str{STRIDE}")  # todo inconsistent naming
if not cropped_dir.exists() and not patches_dir.exists():
    raise FileNotFoundError(
        "Make sure you've preprocessed your imagery with this configuration:"
        f"\n{STRIDE = } \n{PATCH_SIZE = }"
    )

test_patches = list(patches_dir.glob("*.jpg"))

test_patch = test_patches[3]
cropped_image_file = get_cropped_path(test_patch, cropped_dir, STRIDE)
assert cropped_image_file.exists()

image = io.imread(test_patch)
baseline_image = io.imread(cropped_image_file)

# metrics
mean_squared = mean_squared_error(image, baseline_image)
# ssim = structural_similarity(image, baseline_image.squeeze())  # needs to be grayscale
psnr = peak_signal_noise_ratio(image, baseline_image)

fig, ax = plt.subplots(3, 2, figsize=(8, 13))
ax[0, 0].imshow(image[:, :, 0], cmap='jet')
ax[0, 1].imshow(baseline_image[:, :, 0], cmap='jet')
ax[1, 0].imshow(image)
ax[1, 1].imshow(baseline_image)

ax[2, 0].imshow(image[h:w, h:w])
ax[2, 1].imshow(baseline_image[h:w, h:w])
fig.suptitle(f"{STRIDE=} {PATCH_SIZE=}\n MSE: {mean_squared:.2f}, PSNR {psnr:.2f}dB")
plt.tight_layout()

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('Using cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print('Using mps')
else:
    device = torch.device('cpu')
    print('Using cpu')

Using mps


In [10]:
def PSNRLoss(batch_1, batch_2):
    """peak signal-to-noise ratio loss"""
    mse = torch.nn.MSELoss()
    mse_loss = mse(batch_1, batch_2)
    psnr = 10 * torch.log10(1 / mse_loss)
    return psnr

In [11]:
losses = dict(
    mse=torch.nn.MSELoss(),
    bce=torch.nn.BCELoss(),
    psnr=PSNRLoss,
    # maybe add DiceLoss
)

In [12]:
full_set = 200  # running oom for 1000
data_set = Data(data_dir=DATA_DIR, n_images=full_set, load_baseline=True)

loading images:   0%|          | 0/500 [00:00<?, ?it/s]/Users/roetzermatthias/Documents/studies/semester1/02506aia/super_resolution/data.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_images.append(torch.tensor(cropped_image, dtype=torch.float32))
loading images: 100%|██████████| 500/500 [00:04<00:00, 107.75it/s]


In [14]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    logger.info('Using cuda')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    logger.info('Using mps')
else:
    device = torch.device('cpu')
    logger.info('Using cpu')

2023-04-30 15:45:17.393 | INFO     | __main__:<module>:6 - Using mps


In [15]:
validation_split = .2  # percent we want to use for validation
shuffle_dataset = True
random_seed = 42
batch_size = 10

In [16]:
indices = list(range(full_set))
split = int(np.floor(validation_split * full_set))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_loader = torch.utils.data.DataLoader(
    Subset(data_set, train_indices),
    batch_size=batch_size,
    drop_last=True,
)
validation_loader = torch.utils.data.DataLoader(
    Subset(data_set, val_indices),
    batch_size=batch_size * 2,
)

In [17]:
#  ---- init model settings and models ----

In [18]:
# to tune the complexity of the generator
# and thus the complexity of the images
latent_space = 100

lr = 0.0001
nr_epochs = 25
n_filters = 8

batch_losses = []
epoch_losses = []
val_losses = []

image = data_set.input_images[3]

discriminator = VGG16Discriminator().to(device)
generator = UNet(n_filters=n_filters).to(device)

content_loss = torch.nn.L1Loss()  # or torch.nn.MSELoss()
adversarial_loss = torch.nn.BCELoss()

discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))

losses_df = pd.DataFrame(
    index=range(nr_epochs),
    columns=["d_real", "d_fake", "discr", "gen", "gen_val", "ssim", "mse", "psnr"]
)

save_dir = init_save_dir()
logger.info(
    f"\nRunning training with "
    f"\n{nr_epochs=} \n{batch_size=}\n{lr=}\n{n_filters=}"
    f"\nSaving results to:"
    f"\n{save_dir}"
)

2023-04-30 15:45:18.400 | INFO     | __main__:<module>:31 - 
Running training for 15 epochs using
Saving results to:
runs/unet_15453004_eyeliner_reciprocity


In [ ]:


for epoch in range(nr_epochs):
    # for n_batch, (input_batch, target_batch) in tqdm(enumerate(train_loader), desc=f"Epoch {epoch}/{nr_epochs}"):
    for n_batch, (_, target_batch, input_batch) in tqdm(enumerate(train_loader), desc=f"Epoch {epoch}/{nr_epochs}"):
        input_batch = input_batch.to(device)
        target_batch = target_batch.to(device)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        discriminator_optimizer.zero_grad()

        real_outputs = discriminator(target_batch)
        real_loss = adversarial_loss(real_outputs, real_labels)
        losses_df.loc[epoch, "d_real"] = real_loss.item()

        fake_images = generator(input_batch)
        fake_outputs = discriminator(fake_images.detach())
        fake_loss = adversarial_loss(fake_outputs, fake_labels)
        losses_df.loc[epoch, "d_fake"] = fake_loss.item()

        discriminator_loss = (real_loss + fake_loss) / 2
        discriminator_loss.backward()
        discriminator_optimizer.step()
        losses_df.loc[epoch, "discr"] = discriminator_loss.item()

        # Train the generator
        generator_optimizer.zero_grad()
        content_outputs = content_loss(generator(input_batch), target_batch)
        adversarial_outputs = adversarial_loss(
            discriminator(generator(input_batch)), real_labels
        )
        generator_loss = content_outputs + adversarial_outputs
        generator_loss.backward()
        generator_optimizer.step()
        losses_df.loc[epoch, "gen"] = generator_loss.item()

        # logging
        '\t'.join([f"{k}={v:.3f}" for k, v in losses_df.loc[epoch].to_dict().items()])
        # epoch_loss += loss.item()
        batch_loss = losses_df.loc[epoch].mean()
        batch_losses.append(batch_loss)
    epoch_loss = losses_df.loc[epoch].mean()
    epoch_losses.append(epoch_loss / len(train_loader))
    # logger.info(f'Epoch {epoch}/{nr_epochs}, loss {epoch_losses[-1]}')

    with torch.no_grad():
        logits = generator(image.unsqueeze(0).to(device))
        val_loss = 0
        # for val_image_batch, val_target_batch in validation_loader:
        for _, val_target_batch, val_image_batch in validation_loader:
            val_image_batch = val_image_batch.to(device)
            val_target_batch = val_target_batch.to(device)

            logits_batch = generator(val_image_batch)
            loss = content_loss(logits_batch, val_target_batch)
            val_loss += loss.item()
        val_losses.append(val_loss / len(validation_loader))
        losses_df.loc[epoch, "gen_val"] = val_loss
        losses_df.loc[epoch, ["ssim", "mse", "psnr"]] = compare_batches(
            input_batch=torch.nn.functional.tanh(
                torch.nn.functional.relu(logits_batch)
            ),
            target_batch=val_target_batch
        )
    print(
        f"\t Losses: \t" + " | ".join(
            [f"{k}={v:06.3f}" for k, v in losses_df.loc[epoch].to_dict().items()])
    )
    if epoch % 5 == 4:
        logits = generator.relu(logits)
        prob = torch.nn.functional.tanh(logits)

        fig, ax = plt.subplots(1, 2, figsize=(10, 5))

        ax[0].imshow(prob[0, 1].cpu().detach())
        ax[0].imshow(prob.cpu().numpy().squeeze().transpose(1, 2, 0))
        ax[0].set_title(f'Prediction, epoch:{epoch}')

        sns.lineplot(
            losses_df[["gen", "gen_val"]],  # "ssim", "mse"
            ax=ax[1],
            markers=True
        )
        # plt.show(block=False)
        plt.savefig(save_dir / f"loss_epoch{epoch:03}.jpg", dpi=300)
# End of training
torch.save(generator.state_dict(), save_dir / "model.pt")

In [ ]:
# export epoch and validation loss as a csv
pd.DataFrame(
    [epoch_losses, val_losses],
    columns=range(nr_epochs), index=["train", "test"]
).T.to_csv(save_dir / "losses.csv")

In [ ]:

# plotting an inference example
peek_index = 9

input_image = input_batch[peek_index]
target_image = target_batch[peek_index]

logits = generator(input_image.unsqueeze(0).to(device))
reconstruction = torch.nn.functional.tanh(logits).cpu().detach().numpy().squeeze().transpose(1, 2, 0)

input_display = input_image.cpu().numpy().squeeze().transpose(1, 2, 0)
target_display = target_image.cpu().numpy().squeeze().transpose(1, 2, 0)

fig, ax = plt.subplots(2, 3, figsize=(15, 10))
ax[0, 0].imshow(reconstruction[100:200, 100:200, :])
ax[0, 1].imshow(input_display[100:200, 100:200, :])
ax[0, 2].imshow(target_display[100:200, 100:200, :])

ax[1, 0].imshow(reconstruction[:, :, :])
ax[1, 1].imshow(input_display[:, :, :])
ax[1, 2].imshow(target_display[:, :, :])
plt.savefig(save_dir / "reconstruction_sample.jpg", dpi=300)

inferenece_dir = save_dir / "inference"
inferenece_dir.mkdir()

In [ ]:
peek_index = 9

input_image = input_batch[peek_index]
target_image = target_batch[peek_index]

logits = generator(input_image.unsqueeze(0).to(device))
reconstruction = torch.nn.functional.tanh(logits).cpu().detach().numpy().squeeze().transpose(1, 2, 0)

input_display = input_image.cpu().numpy().squeeze().transpose(1, 2, 0)
target_display = target_image.cpu().numpy().squeeze().transpose(1, 2, 0)

fig, ax = plt.subplots(2, 3, figsize=(15, 10))
ax[0, 0].imshow(reconstruction[100:200, 100:200, :])
ax[0, 1].imshow(input_display[100:200, 100:200, :])
ax[0, 2].imshow(target_display[100:200, 100:200, :])

ax[1, 0].imshow(reconstruction[:, :, :])
ax[1, 1].imshow(input_display[:, :, :])
ax[1, 2].imshow(target_display[:, :, :])
plt.savefig()

In [ ]:
!cp -r./runs/ "/content/drive/My Drive/advanced_image_analysis/super_resolution/"